In [4]:
!nvidia-smi

Wed Jul  3 15:08:35 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.239.06   Driver Version: 470.239.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import tensorflow
import matplotlib
import seaborn 
import numpy 
import pandas
import sklearn

print(tensorflow.__version__)
print(matplotlib.__version__)
print(seaborn.__version__)
print(numpy.__version__)
print(pandas.__version__)
print(sklearn.__version__)

2.6.0
3.4.3
0.11.2
1.21.4
1.3.3
1.0


# 1. 데이터 가져오기 & 벡터화 하기

In [6]:
from tensorflow.keras.datasets import reuters
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#빈도수 3000/5000/10000/None 테스트
# num_words 값을 리스트로 설정
num_words_list = [3000, 5000, 10000, None]

#데이터 저장을 위한 딕셔너리
results = {}

for num_words in num_words_list:
    # 데이터 로드 및 분할
    (x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=num_words, 
                                                             test_split=0.2)
    
    #로이터 뉴스 데이터 word index
    word_index = reuters.get_word_index(path="reuters_word_index.json")
    
    #index_to_word
    index_to_word = { index+3 : word for word, index in word_index.items() }
    for index, token in enumerate(("<pad>", "<sos>", "<unk>")):
          index_to_word[index]=token
            
    #텍스트 데이터로 변환
    decoded = []
    for i in range(len(x_train)):
        t = ' '.join([index_to_word[index] for index in x_train[i]])
        decoded.append(t)
    x_train = decoded
    
    decoded = []
    for i in range(len(x_test)):
        t = ' '.join([index_to_word[index] for index in x_test[i]])
        decoded.append(t)

    x_test = decoded

    #DTM 생성
    dtmvector = CountVectorizer()
    x_train_dtm = dtmvector.fit_transform(x_train)
    x_test_dtm = dtmvector.fit_transform(x_test)
    
    #TF-IDF Matrix
    tfidf_transformer = TfidfTransformer()
    train_tfidfv = tfidf_transformer.fit_transform(x_train_dtm)
    test_tfidfv = tfidf_transformer.fit_transform(x_test_dtm)
    
    #딕셔너리에 저장
    results[num_words] = {
        'x_train': train_tfidfv,  
        'y_train': y_train,  
        'x_test': test_tfidfv,               
        'y_test': y_test             
    }

In [7]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 모델 리스트와 이름
models = [LogisticRegression(random_state=0), 
          LinearSVC(random_state=0), 
          RandomForestClassifier(random_state=0)]
model_names = ['Logistic Regression', 'Linear SVM', 'Random Forest']

# 결과 저장할 데이터프레임 생성
results_df = pd.DataFrame(columns=['num_words', 'model', 'accuracy'])

# 각 num_words에 대해 모델별로 학습 및 평가
for num_words in results.keys():
    x_train, y_train = results[num_words]['x_train'], results[num_words]['y_train']
    x_test, y_test = results[num_words]['x_test'], results[num_words]['y_test']
    
    for model, model_name in zip(models, model_names):
        # 모델 학습
        model.fit(x_train, y_train)
        
        # 예측 및 정확도 계산
        y_pred = model.predict(x_test)
        accuracy = accuracy_score(y_test, y_pred)
        
        # 결과 데이터프레임에 추가
        results_df = results_df.append({
            'num_words': num_words,
            'model': model_name,
            'accuracy': accuracy
        }, ignore_index=True)

# 결과 데이터프레임 출력
print(results_df)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ValueError: X has 2911 features, but LogisticRegression is expecting 2919 features as input.